In [1]:
from MZPackage.ModelBuilder import ModelBuilder
from MZPackage.Utilities import *

In [2]:
# Create ModelBuilder object
dataSet = loadData("normalized_data_set")
mb = ModelBuilder()

### PCA

In [ ]:
modelType = "pca"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
codeLayerSize = 50
[model, validatoinLoss, numOfEpochs,_] = mb.trainPca(codeLayerSize, trainingSet, trainingSet, validationSet, validationSet)
model.save('Trained_Model/pca.h5')

### Fully Connected

In [ ]:
modelType = "fullyConnected"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
numOfHiddenLayers = 1
codeLayerSize = 40 # 64
[model, validatoinLoss, numOfEpochs, history] = mb.trainFullyConnected(numOfHiddenLayers, codeLayerSize, trainingSet, trainingSet, validationSet, validationSet, return_best = 1)
model.save('Trained_Model/autoencoder.h5')

### 1D Conv

In [ ]:
modelType = "conv"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
numOfLayers = 2
filtersCountParam = 48
[model, validatoinLoss, numOfEpochs, history] = mb.trainConv(numOfLayers, trainingSet, trainingSet, validationSet, validationSet, filtersCountInFirstLayer = filtersCountParam, filterSize = 4)
#model.save('Trained_Model/conv.h5')

### LSTM 

In [ ]:
modelType = "lstm"
trainingSet, _, validationSet = getReshapedDataSetThreeSplits(dataSet, modelType)
numOfLayers = 1
numOfNeurons = 20 
[model, validatoinLoss, numOfEpochs, history] = mb.trainLstm(numOfLayers, numOfNeurons, trainingSet, trainingSet, validationSet, validationSet)
#model.save('Trained_Model/lstm.h5')